In [1]:
#from TunerFish_utils import load_metadata, extract, get_genre_for_current_song
from TunerFish_utils import *
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

Using Theano backend.


In [2]:
AUDIO_DIR = "D:\\fma_test" # directory for fma music

# location of the track metadata that came along with the fma dataset
TRACK_METADATA = "D:\\fma_metadata\\tracks.csv" 

# location of the genre metadata that came along with the fma dataset
GENRE_METADATA = "D:\\fma_metadata\\genres.csv"

In [3]:
# loading metadata into data frames
tracks = load_metadata(TRACK_METADATA)
genres = load_metadata(GENRE_METADATA)

C:\Users\harrykunx2\Desktop\TunerFish\Classifier\TunerFish_utils.py:40: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  'category', categories=SUBSETS, ordered=True)


In [4]:
# getting musical features and headings for those features
features, heading = extract(tracks, genres, AUDIO_DIR)

Reading Song#1: D:\fma_test\000\000002.mp3...
Hip-Hop
Reading Song#2: D:\fma_test\000\000005.mp3...
Hip-Hop
Reading Song#3: D:\fma_test\000\000010.mp3...
Pop
Reading Song#4: D:\fma_test\000\000140.mp3...
Folk
Reading Song#5: D:\fma_test\000\000141.mp3...
Folk
Reading Song#6: D:\fma_test\000\000148.mp3...
Experimental
Disregarding Experimental music!
Number of experimental music: 1
Reading Song#7: D:\fma_test\000\000182.mp3...
Rock
Reading Song#8: D:\fma_test\000\000190.mp3...
Folk
Reading Song#9: D:\fma_test\000\000193.mp3...
Folk
Reading Song#10: D:\fma_test\000\000194.mp3...
Folk
Reading Song#11: D:\fma_test\000\000197.mp3...
Folk
Reading Song#12: D:\fma_test\000\000200.mp3...
Folk
Reading Song#13: D:\fma_test\000\000203.mp3...
Folk
Reading Song#14: D:\fma_test\000\000204.mp3...
Folk
Reading Song#15: D:\fma_test\000\000207.mp3...
Folk
Reading Song#16: D:\fma_test\000\000210.mp3...
Folk
Reading Song#17: D:\fma_test\000\000211.mp3...
Folk
Reading Song#18: D:\fma_test\000\000212.mp3...


Reading Song#121: D:\fma_test\001\001929.mp3...
Experimental
Disregarding Experimental music!
Number of experimental music: 25
Reading Song#122: D:\fma_test\001\001930.mp3...
Rock
Reading Song#123: D:\fma_test\002\002012.mp3...
Experimental
Disregarding Experimental music!
Number of experimental music: 26
Reading Song#124: D:\fma_test\002\002096.mp3...
Experimental
Disregarding Experimental music!
Number of experimental music: 27
Reading Song#125: D:\fma_test\002\002097.mp3...
Folk
Reading Song#126: D:\fma_test\002\002099.mp3...
Pop
Reading Song#127: D:\fma_test\003\003263.mp3...
Folk
Reading Song#128: D:\fma_test\003\003264.mp3...
Folk
Reading Song#129: D:\fma_test\003\003265.mp3...
Folk
Reading Song#130: D:\fma_test\003\003266.mp3...
Folk
Reading Song#131: D:\fma_test\003\003270.mp3...
Folk
Reading Song#132: D:\fma_test\003\003271.mp3...
Folk
Reading Song#133: D:\fma_test\003\003272.mp3...
Folk
Reading Song#134: D:\fma_test\003\003273.mp3...
Folk
Reading Song#135: D:\fma_test\003\003

Reading Song#263: D:\fma_test\005\005376.mp3...
Folk
Reading Song#264: D:\fma_test\005\005381.mp3...
Folk
Reading Song#265: D:\fma_test\005\005521.mp3...
Experimental
Disregarding Experimental music!
Number of experimental music: 28
Reading Song#266: D:\fma_test\005\005879.mp3...
Folk
Reading Song#267: D:\fma_test\005\005936.mp3...
Folk
Reading Song#268: D:\fma_test\005\005940.mp3...
Folk
Reading Song#269: D:\fma_test\006\006329.mp3...
International
Reading Song#270: D:\fma_test\006\006330.mp3...
International
Reading Song#271: D:\fma_test\006\006331.mp3...
International
Reading Song#272: D:\fma_test\006\006332.mp3...
International
Reading Song#273: D:\fma_test\006\006333.mp3...
International
Reading Song#274: D:\fma_test\006\006342.mp3...
International
Reading Song#275: D:\fma_test\006\006354.mp3...
International
Reading Song#276: D:\fma_test\006\006357.mp3...
International
Reading Song#277: D:\fma_test\006\006358.mp3...
International
Reading Song#278: D:\fma_test\006\006360.mp3...
In

Reading Song#379: D:\fma_test\008\008256.mp3...
Folk
Reading Song#380: D:\fma_test\008\008259.mp3...
Folk
Reading Song#381: D:\fma_test\008\008261.mp3...
Folk
Reading Song#382: D:\fma_test\008\008345.mp3...
Experimental
Disregarding Experimental music!
Number of experimental music: 51
Reading Song#383: D:\fma_test\008\008357.mp3...
Experimental
Disregarding Experimental music!
Number of experimental music: 52
Reading Song#384: D:\fma_test\008\008363.mp3...
Experimental
Disregarding Experimental music!
Number of experimental music: 53
Reading Song#385: D:\fma_test\008\008372.mp3...
Experimental
Disregarding Experimental music!
Number of experimental music: 54
Reading Song#386: D:\fma_test\008\008416.mp3...
Experimental
Disregarding Experimental music!
Number of experimental music: 55
Reading Song#387: D:\fma_test\009\009152.mp3...
Experimental
Disregarding Experimental music!
Number of experimental music: 56
Reading Song#388: D:\fma_test\009\009155.mp3...
Experimental
Disregarding Exper

Reading Song#490: D:\fma_test\011\011268.mp3...
Rock
Reading Song#491: D:\fma_test\011\011298.mp3...
Hip-Hop
Reading Song#492: D:\fma_test\011\011299.mp3...
Hip-Hop
Reading Song#493: D:\fma_test\011\011306.mp3...
Rock
Reading Song#494: D:\fma_test\011\011333.mp3...
Experimental
Disregarding Experimental music!
Number of experimental music: 82
Reading Song#495: D:\fma_test\011\011334.mp3...
Experimental
Disregarding Experimental music!
Number of experimental music: 83
Reading Song#496: D:\fma_test\011\011503.mp3...
Folk
Reading Song#497: D:\fma_test\011\011504.mp3...
Folk
Reading Song#498: D:\fma_test\011\011505.mp3...
Folk
Reading Song#499: D:\fma_test\011\011508.mp3...
Folk
Reading Song#500: D:\fma_test\011\011544.mp3...
Experimental
Disregarding Experimental music!
Number of experimental music: 84
Reading Song#501: D:\fma_test\011\011638.mp3...
Instrumental
Reading Song#502: D:\fma_test\011\011671.mp3...
International
Reading Song#503: D:\fma_test\011\011672.mp3...
International
Read

Reading Song#631: D:\fma_test\012\012986.mp3...
Folk
Reading Song#632: D:\fma_test\013\013191.mp3...
Pop
Reading Song#633: D:\fma_test\013\013197.mp3...
Pop
Reading Song#634: D:\fma_test\013\013199.mp3...
Hip-Hop
Reading Song#635: D:\fma_test\013\013201.mp3...
Hip-Hop
Reading Song#636: D:\fma_test\013\013218.mp3...
Pop
Reading Song#637: D:\fma_test\013\013220.mp3...
Pop
Reading Song#638: D:\fma_test\013\013325.mp3...
Rock
Reading Song#639: D:\fma_test\013\013328.mp3...
Pop
Reading Song#640: D:\fma_test\013\013362.mp3...
Pop
Reading Song#641: D:\fma_test\013\013378.mp3...
Pop
Reading Song#642: D:\fma_test\013\013474.mp3...
Electronic
Reading Song#643: D:\fma_test\013\013537.mp3...
International
Reading Song#644: D:\fma_test\013\013538.mp3...
International
Reading Song#645: D:\fma_test\013\013539.mp3...
International
Reading Song#646: D:\fma_test\013\013540.mp3...
International
Reading Song#647: D:\fma_test\013\013556.mp3...
Rock
Reading Song#648: D:\fma_test\013\013561.mp3...
Hip-Hop
Re

Reading Song#759: D:\fma_test\016\016162.mp3...
Hip-Hop
Reading Song#760: D:\fma_test\016\016163.mp3...
Pop
Reading Song#761: D:\fma_test\016\016334.mp3...
International
Reading Song#762: D:\fma_test\016\016337.mp3...
Pop
Reading Song#763: D:\fma_test\016\016338.mp3...
Pop
Reading Song#764: D:\fma_test\016\016339.mp3...
Pop
Reading Song#765: D:\fma_test\016\016340.mp3...
Pop
Reading Song#766: D:\fma_test\016\016354.mp3...
Rock
Reading Song#767: D:\fma_test\016\016743.mp3...
International
Reading Song#768: D:\fma_test\016\016744.mp3...
International
Reading Song#769: D:\fma_test\016\016745.mp3...
International
Reading Song#770: D:\fma_test\016\016747.mp3...
International
Reading Song#771: D:\fma_test\016\016819.mp3...
Rock
Reading Song#772: D:\fma_test\016\016820.mp3...
Rock
Reading Song#773: D:\fma_test\016\016821.mp3...
Rock
Reading Song#774: D:\fma_test\016\016822.mp3...
Rock
Reading Song#775: D:\fma_test\016\016878.mp3...
Hip-Hop
Reading Song#776: D:\fma_test\016\016879.mp3...
Hip-H

In [5]:
# creating a data frame from features and heading
songs_df = pd.DataFrame(features, columns=heading)
songs_df

,id,songname,zcr_mean,zcr_std,s_cen_mean,s_cen_std,s_con_mean,s_con_std,s_band_mean,s_band_std,...,mfcss9_std,mfcss10_mean,mfcss10_std,mfcss11_mean,mfcss11_std,mfcss12_mean,mfcss12_std,mfcss13_mean,mfcss13_std,genre
0,1,D:\fma_test\000\000002.mp3,0.163720,0.093397,3037.646484,990.452397,19.196282,6.457959,2690.117840,357.412573,...,8.723731,-10.330374,9.556734,3.406905,10.555513,-3.014135,8.787369,8.196358,7.852417,Hip-Hop
1,2,D:\fma_test\000\000005.mp3,0.100103,0.066566,2414.503249,870.258029,19.144967,6.997129,2598.633925,404.213975,...,10.152660,-1.835227,9.910918,8.863636,8.553726,3.440012,9.489809,7.301098,7.502436,Hip-Hop
2,3,D:\fma_test\000\000010.mp3,0.148673,0.028250,2349.564320,376.768810,20.867279,6.762833,2051.910358,340.660599,...,6.636336,-9.246256,6.882463,2.240332,5.936033,-6.418429,6.014394,2.678393,5.566027,Pop
3,4,D:\fma_test\000\000140.mp3,0.044345,0.051895,1746.271125,1024.416550,20.436153,6.011029,2484.461469,577.014901,...,10.756562,-4.780853,9.297045,-1.614263,7.863483,-6.837076,8.229039,2.880230,7.975400,Folk
4,5,D:\fma_test\000\000141.mp3,0.061877,0.051724,1654.240257,800.220223,22.923613,6.527027,2036.525423,543.562709,...,7.817021,-3.327126,8.208024,-3.396589,8.801828,-4.810521,7.362343,3.130622,8.733100,Folk
5,7,D:\fma_test\000\000182.mp3,0.100565,0.025230,1876.797169,361.376338,22.232775,8.703256,1749.208960,273.550070,...,7.375034,12.132687,6.978716,12.425987,6.565393,3.194551,6.676619,11.543197,6.108296,Rock
6,8,D:\fma_test\000\000190.mp3,0.088393,0.047448,1899.920558,701.756127,25.281517,6.566406,2019.028214,356.298362,...,8.944339,-16.557027,8.302337,-4.756800,9.124164,-8.715330,9.825579,-6.131213,9.514558,Folk
7,9,D:\fma_test\000\000193.mp3,0.077351,0.038008,1584.312861,623.072142,26.349444,7.029009,1754.518587,417.037931,...,8.700961,-11.734615,7.633208,-4.999176,8.127394,-7.849339,8.298889,-7.091806,7.323960,Folk
8,10,D:\fma_test\000\000194.mp3,0.072886,0.029054,1546.779400,488.450658,26.223748,6.793024,1760.761918,359.394628,...,8.041008,-13.380765,8.337683,-2.445210,7.569080,-8.818468,6.812469,-9.325425,7.743841,Folk
9,11,D:\fma_test\000\000197.mp3,0.054117,0.028980,1143.168784,452.627163,25.603666,6.808681,1364.550353,329.444707,...,7.045476,-7.864277,6.971336,-7.552975,7.784744,-5.776649,8.502081,-7.707771,8.928392,Folk


**Labels**

In [7]:
songs_labels.value_counts()

Folk             194
International    167
Hip-Hop          132
Rock             126
Pop               90
Electronic        33
Instrumental      31
Name: genre, dtype: int64

In [9]:
len(songs_features.columns.value_counts())

36

**Random forest classifier, normalized data**

In [33]:
# separate the features and the label
songs_features = get_features(songs_df, "zcr_mean", "mfcss13_std")
songs_labels = get_labels(songs_df, "genre")

In [34]:
# normalize the features
normalized_songs_features = normalize(songs_features)

In [35]:
# converting labels into categorical data and converting to numpy array
Y = to_categorical(songs_labels)

In [36]:
# converting features to numpy array
X = transform_to_matrix(normalized_songs_features)

In [48]:
# Splitting training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)

In [49]:
# Creating a random forest classifier
rclf = RandomForestClassifier(n_estimators=150)
rclf.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=150, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [50]:
rclf.score(X_test, Y_test)

0.62109375

**Confusion matrix**

In [16]:
pred = rclf.predict(X_test)

In [17]:
from sklearn.metrics import confusion_matrix

In [18]:
confusion_matrix(Y_test, pred)

array([[ 0,  0,  4,  0,  0,  0,  2],
       [ 0, 51,  2,  0, 10,  1,  3],
       [ 0,  1, 36,  0,  4,  2,  2],
       [ 0,  9,  0,  0,  2,  0,  0],
       [ 0,  9,  2,  0, 40,  0,  5],
       [ 0,  3,  5,  0,  8,  6,  7],
       [ 0,  6,  3,  0,  4,  1, 28]])

**Random forest classifier, non-normalized data**

In [30]:
# separate the features and the label
songs_features = get_features(songs_df, "zcr_mean", "mfcss13_std")
songs_labels = get_labels(songs_df, "genre")

# converting labels into categorical data and converting to numpy array
Y = to_categorical(songs_labels)

# converting features to numpy array
X = transform_to_matrix(songs_features)

In [31]:
# Splitting training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)

# Creating a random forest classifier
rclf = RandomForestClassifier(n_estimators=150)
rclf.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=150, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [32]:
rclf.score(X_test, Y_test)

0.6328125